In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib as mpl
from sklearn.preprocessing import MinMaxScaler

In [2]:
garm_data = pd.read_csv('garments_worker_productivity.csv')

In [3]:
garm_data

,date,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,Thursday,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,Thursday,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,Thursday,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,Thursday,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,Thursday,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,3/11/2015,Quarter2,finishing,Wednesday,10,0.75,2.90,NaN,960,0,0.0,0,0,8.0,0.628333
1193,3/11/2015,Quarter2,finishing,Wednesday,8,0.70,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1194,3/11/2015,Quarter2,finishing,Wednesday,7,0.65,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1195,3/11/2015,Quarter2,finishing,Wednesday,9,0.75,2.90,NaN,1800,0,0.0,0,0,15.0,0.505889


In [4]:
#Dropping columns with null values or non contributing values
garm_data_input = garm_data.drop('wip', axis = 1)
garm_data_input = garm_data_input.drop('date', axis = 1)
garm_data_input = garm_data_input.drop('actual_productivity', axis = 1)
garm_data_output = garm_data[['actual_productivity']]

In [5]:
#Encoding columns 
columnsToEncode = ['quarter', 'department', 'day']

for col in columnsToEncode:
    distinct_values = garm_data_input[col].unique()
    mapping_dict = dict(zip(distinct_values, range(1, len(distinct_values) + 1)))
    garm_data_input[col] = garm_data_input[col].replace(mapping_dict)

In [6]:
scaler = MinMaxScaler()
# transform data
garm_data_inputs = scaler.fit_transform(garm_data_input)

# Initial test to change values in single column

distinctVal = garm_data['quarter'].unique()
map_distinctVal = dict(zip(distinctVal, range(1, len(distinctVal)+1)))
garm_data['quarter'] = garm_data['quarter'].replace(map_distinctVal)

In [7]:
garm_data_inputs

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.65517241],
       [0.        , 0.5       , 0.        , ..., 0.        , 0.        ,
        0.06896552],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.32758621],
       ...,
       [0.25      , 1.        , 1.        , ..., 0.        , 0.        ,
        0.06896552],
       [0.25      , 1.        , 1.        , ..., 0.        , 0.        ,
        0.14942529],
       [0.25      , 1.        , 1.        , ..., 0.        , 0.        ,
        0.04597701]])

In [8]:
garm_data_output

,actual_productivity
0,0.940725
1,0.886500
2,0.800570
3,0.800570
4,0.800382
...,...
1192,0.628333
1193,0.625625
1194,0.625625
1195,0.505889


# Creating the sigmoid activation function and sigmoid derivative

In [9]:
# This is the sigmoid Function
def sigmoid(sum):
  return 1 / (1 + np.exp(-sum))

#This is the sigmoid derivative as used before
def sigmoid_derivative(sigmoid):
  return sigmoid * (1 - sigmoid)

In [10]:
inputs = garm_data_input.to_numpy()

In [11]:
output = garm_data_output.to_numpy()

In [12]:
inputs.shape

(1197, 12)

In [13]:
output.shape

(1197, 1)

In [14]:
hidden_neurons = 20

In [15]:
#Creating random wieghts for all features
weights_hidden = np.random.rand(inputs.shape[1], hidden_neurons)*2 - 1

In [16]:
weights_hidden.shape

(12, 20)

In [17]:
weights_output = np.random.rand(hidden_neurons, 1) * 2 - 1

In [18]:
weights_output.shape

(20, 1)

#Defining model
epochs = 1000
learning_rate = 0.1
error = []

for epoch in range(epochs):
  input_layer = inputs
  sum_synapse0 = np.dot(input_layer, weights_hidden)
  hidden_layer = sigmoid(sum_synapse0)

  sum_synapse1 = np.dot(hidden_layer, weights_output)
  output_layer = sigmoid(sum_synapse1)

  error_output_layer = output - output_layer
  average = np.mean(abs(error_output_layer))
    #print after every specified range of the value
  if epoch % 25 == 0:
    print('Epoch: ' + str(epoch + 1) + ' Error: ' + str(average))
    error.append(average)
  
  derivative_output = sigmoid_derivative(output_layer)
  delta_output = error_output_layer * derivative_output
  
  weights1T = weights_output.T
  delta_output_weight = delta_output.dot(weights1T)
  delta_hidden_layer = delta_output_weight * sigmoid_derivative(hidden_layer)
  
  hidden_layerT = hidden_layer.T
  input_x_delta1 = hidden_layerT.dot(delta_output)
  weights_1 = weights_output + (input_x_delta1 * learning_rate)
  
  input_layerT = input_layer.T
  input_x_delta0 = input_layerT.dot(delta_hidden_layer)
  weights_0 = weights_hidden + (input_x_delta0 * learning_rate)

In [21]:
epochs = 1000
learning_rate = 0.01
error = []

for epoch in range(epochs):
    input_layer = inputs
    sum_synapse0 = np.dot(input_layer, weights_hidden)
    hidden_layer = sigmoid(sum_synapse0)

    sum_synapse1 = np.dot(hidden_layer, weights_output)
    output_layer = sigmoid(sum_synapse1)

    error_output_layer = output - output_layer
    average = np.mean(abs(error_output_layer))

    # Print error and append to list after each epoch
    print('Epoch:', epoch + 1, 'Error:', average)
    error.append(average)

    # Backpropagation and weight updates
    derivative_output = sigmoid_derivative(output_layer)
    delta_output = error_output_layer * derivative_output

    weights1T = weights_output.T
    delta_output_weight = delta_output.dot(weights1T)
    delta_hidden_layer = delta_output_weight * sigmoid_derivative(hidden_layer)

    hidden_layerT = hidden_layer.T
    input_x_delta1 = hidden_layerT.dot(delta_output)
    weights_output += (input_x_delta1 * learning_rate)  # Update weights_output directly

    input_layerT = input_layer.T
    input_x_delta0 = input_layerT.dot(delta_hidden_layer)
    weights_hidden += (input_x_delta0 * learning_rate)  # Update weights_hidden directly


C:\Users\Rhodd\AppData\Local\Temp\ipykernel_1676\2690463717.py:3: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-sum))


Epoch: 1 Error: 0.26616996958016537
Epoch: 2 Error: 0.26616996958016537
Epoch: 3 Error: 0.26616996958016537
Epoch: 4 Error: 0.26616996958016537
Epoch: 5 Error: 0.26616996958016537
Epoch: 6 Error: 0.26616996958016537
Epoch: 7 Error: 0.26616996958016537
Epoch: 8 Error: 0.26616996958016537
Epoch: 9 Error: 0.26616996958016537
Epoch: 10 Error: 0.26616996958016537
Epoch: 11 Error: 0.26616996958016537
Epoch: 12 Error: 0.26616996958016537
Epoch: 13 Error: 0.26616996958016537
Epoch: 14 Error: 0.26616996958016537
Epoch: 15 Error: 0.26616996958016537
Epoch: 16 Error: 0.26616996958016537
Epoch: 17 Error: 0.26616996958016537
Epoch: 18 Error: 0.26616996958016537
Epoch: 19 Error: 0.26616996958016537
Epoch: 20 Error: 0.26616996958016537
Epoch: 21 Error: 0.26616996958016537
Epoch: 22 Error: 0.26616996958016537
Epoch: 23 Error: 0.26616996958016537
Epoch: 24 Error: 0.26616996958016537
Epoch: 25 Error: 0.26616996958016537
Epoch: 26 Error: 0.26616996958016537
Epoch: 27 Error: 0.26616996958016537
Epoch: 28 

Epoch: 371 Error: 0.26616996958016537
Epoch: 372 Error: 0.26616996958016537
Epoch: 373 Error: 0.26616996958016537
Epoch: 374 Error: 0.26616996958016537
Epoch: 375 Error: 0.26616996958016537
Epoch: 376 Error: 0.26616996958016537
Epoch: 377 Error: 0.26616996958016537
Epoch: 378 Error: 0.26616996958016537
Epoch: 379 Error: 0.26616996958016537
Epoch: 380 Error: 0.26616996958016537
Epoch: 381 Error: 0.26616996958016537
Epoch: 382 Error: 0.26616996958016537
Epoch: 383 Error: 0.26616996958016537
Epoch: 384 Error: 0.26616996958016537
Epoch: 385 Error: 0.26616996958016537
Epoch: 386 Error: 0.26616996958016537
Epoch: 387 Error: 0.26616996958016537
Epoch: 388 Error: 0.26616996958016537
Epoch: 389 Error: 0.26616996958016537
Epoch: 390 Error: 0.26616996958016537
Epoch: 391 Error: 0.26616996958016537
Epoch: 392 Error: 0.26616996958016537
Epoch: 393 Error: 0.26616996958016537
Epoch: 394 Error: 0.26616996958016537
Epoch: 395 Error: 0.26616996958016537
Epoch: 396 Error: 0.26616996958016537
Epoch: 397 E

Epoch: 752 Error: 0.26616996958016537
Epoch: 753 Error: 0.26616996958016537
Epoch: 754 Error: 0.26616996958016537
Epoch: 755 Error: 0.26616996958016537
Epoch: 756 Error: 0.26616996958016537
Epoch: 757 Error: 0.26616996958016537
Epoch: 758 Error: 0.26616996958016537
Epoch: 759 Error: 0.26616996958016537
Epoch: 760 Error: 0.26616996958016537
Epoch: 761 Error: 0.26616996958016537
Epoch: 762 Error: 0.26616996958016537
Epoch: 763 Error: 0.26616996958016537
Epoch: 764 Error: 0.26616996958016537
Epoch: 765 Error: 0.26616996958016537
Epoch: 766 Error: 0.26616996958016537
Epoch: 767 Error: 0.26616996958016537
Epoch: 768 Error: 0.26616996958016537
Epoch: 769 Error: 0.26616996958016537
Epoch: 770 Error: 0.26616996958016537
Epoch: 771 Error: 0.26616996958016537
Epoch: 772 Error: 0.26616996958016537
Epoch: 773 Error: 0.26616996958016537
Epoch: 774 Error: 0.26616996958016537
Epoch: 775 Error: 0.26616996958016537
Epoch: 776 Error: 0.26616996958016537
Epoch: 777 Error: 0.26616996958016537
Epoch: 778 E